# ÁLGEBRA LINEAR COMPUTACIONAL

## SEGUNDA AVALIAÇÃO - 2020.2 - PROFESSOR ERITO MARQUES   

---
### GRUPO:
**Ícaro Santos Barcelos Pereira - _2017780187_**\
**Leandro Bataglia Pereira - _2018780199_**\
**Pedro Henrique Farolfi Camelo - _2018780326_**

---

Repositório dos algoritmos solicitados na segunda prova da disciplina de Álgebra Linear Computacional.
##### Instruções:
As respostas devem ser enviadas no formato jupyter notebook.\
Os trabalhos devem ser feitos em grupo. Não é permitida troca de informações entre os grupos.\
Fixada a data de entrega, não serão aceitos trabalhos entregues fora dessa data.

In [87]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore",category=UserWarning)

In [13]:
import numpy as np
import pandas as pd
import math
from normas import *
from criterios import *
from cholesky import *
from iterativos import *
from benford import *
from fatoracaoLU import fatoracaoLU
from substituicao import resolve_substituicao
from sympy import Matrix, init_printing
init_printing()
#BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*)

In [14]:
# Coverti isso aqui em um procedimento, a parada é que vc pode mandar isso
# ocorrer no início de qualquer célula e pode alterar o excel em tempo de execução :)

# CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64'
def ler_matriz_A_vetor_b(file_excel, sheet_matriz, sheet_vetor):
    df_matriz = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name=sheet_matriz) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name=sheet_vetor) # Le o arquivo do excel que contém o vetor b sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor

### 1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie:

In [15]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** se A é tridiagonal;

In [16]:
def verifica_tridiagonal(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é tridiagonal")
        return False

    for i in range(linhas):
        for j in range(colunas):
            if np.abs(i-j) <= 1 and matriz_A[i,j] == 0:
                return False
            if np.abs(i-j) >= 2 and matriz_A[i,j] != 0:
                return False
    return True

In [17]:
verifica_tridiagonal(matriz, debug=True)

False

In [18]:
#declarando vetor na mão:
A = np.array([[1,1,0],
              [1,1,1],
              [0,1,1]])
verifica_tridiagonal(A)

True

**b)** se A é triangular superior;

In [19]:
def verifica_tri_superior(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é triangular superior")
        return False

    for i in range(linhas):
        for j in range(i):
            if matriz_A[i,j] != 0:
                return False
    return True

In [20]:
verifica_tri_superior(matriz, debug=True)

False

**c)** se A é triangular inferior;

In [21]:
def verifica_tri_inferior(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        print("A matriz não é quadrada -> não é triangular inferior")
        return False

    for j in range(colunas):
        for i in range(j):
            if matriz_A[i,j] != 0:
                return False
    return True

In [22]:
verifica_tri_inferior(matriz)

False

**d)** se Traço de A é maior que 4.

In [23]:
def traco(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> traço não está definido.")
        return False
    
    n, traco = min((linhas, colunas)), 0
    for i in range(n):
        traco += matriz_A[i,i]
    if traco > 4:
        print("O traço da matriz é " + str(traco) + ", portanto é maior que 4")
        return True
    print("O traço da matriz é " + str(traco) + ", portanto não é maior que 4")
    return False

In [24]:
A = np.array([[1,2,3],[1,2,3]])
traco(A, debug=True)

A matriz não é quadrada -> traço não está definido.


False

In [25]:
traco(matriz, debug=True)

O traço da matriz é 1179.0, portanto é maior que 4


True

---
### 2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

In [26]:
def resolve_substituicao(matriz_A, vetor_b):
    
    if checa_tamanho_matriz_vetor(matriz_A, vetor_b) == False:
        return False
        
    if(verifica_tri_inferior(matriz_A)):
        return resolve_subs_frente(matriz_A, vetor_b)

    elif(verifica_tri_superior(matriz_A)):
        return resolve_subs_tras(matriz_A,vetor_b)
        
    else:
        print("A matriz não é triangular")
        return False
        
        
def checa_tamanho_matriz_vetor(matriz_A, vetor_b):
    lin_A, col_A = np.shape(matriz_A)
    col_b = np.shape(vetor_b)
    
    try:
        assert lin_A == col_b[0]
    except:
        print("A quantidade de linhas em A e de colunas em b são distintas")
        return False
    return True
    
    
def resolve_subs_frente(matriz_A, vetor_b): # Resolve um sistema com matriz triangular inferior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n):
        for j in range(i):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

def resolve_subs_tras(matriz_A,vetor_b): # Resolve um sistema com matriz triangular superior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n-1,-1,-1):
        for j in range(n-1,i,-1):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

In [27]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

delta_x = resolve_substituicao(matriz, vetor)

if delta_x != False:
    print("Vetor x resultado da substituição:\n",delta_x,"\n")

A matriz não é triangular


---
### 3) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:



In [28]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** Avalie se é possível obter o fator de Cholesky;

In [71]:
def checa_positiva_definida(matriz_A, debug=False):
        
    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A
    for i in range(n):
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
            if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
                if debug:
                    print("Erro: A matriz não é positiva definida.")
                return False
            matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
            for j in range(i+1,n): # Não é executado quando i = n
                for k in range(i): # Não é executado quando i = 0
                    matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
                matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]
    return True         

In [30]:
A = np.array([[2.,1.,0.],
              [1.,2.,-1.],
              [0.,-1.,2.]])

In [31]:
A = np.array([[2.,1.,0.],
              [1.,2.,-1.],
              [0.,-1.,2.]])

In [32]:
b = np.array([[3.],[0.],[1.]])

In [33]:
checa_cholesky(A, debug=True)

True

**b)** Resolva o sistema Ax = b usando a fatoração de Cholesky.

In [34]:
def decomposicao_cholesky(matriz_A, vetor_b):
    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A

    for i in range(n):
        for k in range(i): # Não é executado quando i = 0
            matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
        if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
            print("Erro: A matriz não é positiva definida.")
            return
        matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
        for j in range(i+1,n): # Não é executado quando i = n
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
            matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]

    return resolve_cholesky(matriz_R, vetor_b)

def resolve_cholesky(matriz_R, vetor_b):
    matriz_Rt = np.copy(np.transpose(matriz_R)) # matriz_rt é a matriz R transposta
    
    vetor_y = subs.resolve_substituicao(matriz_Rt, vetor_b) # Faz o cálculo de Rt * y = b para obter o vetor y
    vetor_x = subs.resolve_substituicao(matriz_R, vetor_y) # Faz o cálculo de R * x = y para obter o vetor x (resultado)

    return vetor_x # Retorna o vetor x (resultado)

In [35]:
# TESTE DO ALGORITMO DE DECOMPOSIÇÃO DE CHOLESKY E NORMA RESIDUAL (cholesky.py, normas.py)
delta_x = decomposicao_cholesky(matriz, vetor)
print("Vetor x resultado da Decomposição de Cholesky:\n",delta_x,"\n")
#residual(matriz, vetor, delta_x)

Erro: A matriz não é positiva definida.
Vetor x resultado da Decomposição de Cholesky:
 None 



---
### 4) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:

In [36]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** Avalie se algum critério é satisfeito para aplicação dos métodos de Gauss-Seidel, Jacobi e SOR;

In [37]:
def criterio_linhas(matriz_A): # Verificar o critério das linhas para o método de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    soma = 0

    for i in range(n): # Percorre toda a matriz
        for j in range(m):
            if i != j: # Desconsidera os termos da diagonal principal
                soma += np.abs(matriz_A[i,j])

        if soma >= np.abs(matriz_A[i,i]): # Verifica se a soma de todos os termos (sem contar o termo da diagonal principal) de cada linha é menor ou maior que o termo da diagonal principal (Critério das linhas)
            return False

        soma = 0
    return True

In [38]:
def criterio_sassenfeld(matriz_A):
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que auxilia nos somatórios dos termos tanto de Aij*Beta, quanto de Aij
    vetor_betao = np.zeros(m) # Vetor de Betas iniciado com termos iguais a zero

    for i in range(n):
        for j in range(i): # j vai até i-1 (desconsidera a primeira iteração)
            aux += np.abs(matriz_A[i,j]) * vetor_betao[j] # A partir da segunda iteração, inclui a multiplicação de Aij pelos Betas anteriores no somatório (já que na primeira iteração não há Betas anteriores, obviamente)
        for j in range(i,n):
            aux += np.abs(matriz_A[i,j]) # Somatório dos módulos dos termos de Aij
        vetor_betao[i] = aux/np.abs(matriz_A[i,i]) #Divide o valor da soma pelo termo da diagonal principal
        aux = 0 # Zera a variável auxiliar pra próxima iteração
    
    print("Maior betão:", np.max(vetor_betao))
    if np.max(vetor_betao) < 1: # Verifica se a condição do critério de Sassenfeld (max(B) < 1) é satisfeita
        return True
    return False

In [39]:
criterio_linhas(matriz)

False

In [40]:
criterio_sassenfeld(matriz)

Maior betão: 10602274533433.357


False

**b)** Resolva o sistema Ax = b usando os métodos iterativos de Gauss-Seidel, Jacobi e SOR.

## Jacobi

In [41]:
def jacobi(matriz_A, vetor_b, vetor_x=None, N=10000, p=1e-15): # Função que executa o algoritmo de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if vetor_x == None: # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)
    
    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função (N) se a condição de parada não for satisfeita
        for i in range(n):
            for j in range(n):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj
            
            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        for i in range(n):
            if vetor_x[i] != 0 and np.abs(vetor_x[i] - vetor_aux[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p
                print("(Jacobi) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Jacobi) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [42]:
print(jacobi(matriz, vetor, N=100))

ATENÇÃO: O critério das linhas não foi satisfeito.
(Jacobi) Resultado encontrado com o número máximo de iterações.
[-2.21507407e+191 -7.85062201e+191 -4.02063148e+191 -3.00124088e+191
 -2.34581480e+192 -2.32464880e+191 -4.58103429e+192 -4.79832453e+191
 -5.85375866e+191 -4.90112952e+191 -2.01091315e+191 -2.59706719e+191
 -6.26779852e+191 -1.21273532e+192 -8.04504337e+192 -2.42684232e+192
 -4.65327284e+191 -2.98587023e+191 -3.29553950e+191 -4.17899256e+191
 -4.88807065e+191 -4.20548399e+191 -3.16760574e+192 -1.26588784e+192
 -1.08434271e+193]


## Gauss-Seidel

In [95]:
def seidel(matriz_A, vetor_b, vetor_x=None, N=10000, p=1e-15): # Função que executa o algoritmo de Gauss-Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if vetor_x == None: # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1
    	
    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério de Sassenfeld é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0 and np.abs(vetor_x[i] - vetor_aux[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p
                print("(Seidel) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Seidel) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [88]:
print(seidel(matriz, vetor))
#print(sor(matriz, vetor))

ATENÇÃO: O critério das linhas não foi satisfeito.
Maior betão: 10184953.97183485
ATENÇÃO: O critério de Sassenfeld não foi satisfeito.
(Seidel) Resultado encontrado com o número máximo de iterações.
[-21.56775216 -32.54641235  -5.32821735  -2.69145685  20.32386694
  23.40691512  54.42531028  -5.62011583   3.62948958  52.68351969
 -14.87951715  15.85691206 -17.19311452 -15.14191749  -3.79030319
  10.5149987   10.04383099  -4.00756475 -21.51290161   5.40162896
 -39.49506381  -4.8946655   -0.54243828   3.06451631   8.30498115]


## SOR

In [132]:
def calcular_omega_otimo(matriz_A):
    
    if checa_positiva_definida(matriz_A, debug=True) == False or verifica_tridiagonal(matriz_A, debug=True) == False:
        return None        
    
    D = np.copy(matriz_A)
    for i in range(3):
        for j in range(3):
            if i != j:
                D[i,j] = 0.
    D = np.linalg.inv(d)
    
    L = np.tril(A)
    U = np.triu(A)

    for i in range(3):
        for j in range(3):
            if i == j:
                L[i,j] = 0.
                U[i,j] = 0.

    LU = L + U
    T = np.matmul(d, lu)
    P = np.linalg.eig(T)
    p = np.amax(X[0])
    
    print("omega: {}".format(2/(1 + math.sqrt(1 - pow(p, 2)))))
    return 2/(1 + math.sqrt(1 - pow(p, 2)))
    

In [131]:
def sor(matriz_A, vetor_b, vetor_x=None, N=10000, p=1e-15, w=1.7): # Quando w = 1 é igual ao Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj
    
    omega = calcular_omega_otimo(matriz_A)
    
    if omega != None:
        w = omega

    if vetor_x == None: # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (1-w)*vetor_x[i] + (w*(vetor_b[i] - aux))/matriz_A[i,i] # Executa a iteração do método com (1-w)*xi(bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        for i in range(n): # Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0 and np.abs(vetor_x[i] - vetor_aux[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p
                print("(SOR) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(SOR) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [133]:
a = np.array([[2.,1.,0.],[1.,2.,-1.],[0.,-1.,2.]])

In [134]:
b = np.array([[3.],[0.],[1.]])

In [135]:
x = np.array([[1.5],[0.],[0.5]])

In [136]:
sor(a, b, x, N=2)

omega: 1.17157287525381


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [45]:
d = np.copy(a)
for i in range(3):
    for j in range(3):
        if i != j:
            d[i,j] = 0.
d = np.linalg.inv(d)

In [46]:
l = np.tril(a)
u = np.triu(a)

for i in range(3):
    for j in range(3):
        if i == j:
            l[i,j] = 0.
            u[i,j] = 0.
            
lu = l+u

In [76]:
T = np.matmul(d, lu)

In [77]:
T

array([[ 0. ,  0.5,  0. ],
       [ 0.5,  0. , -0.5],
       [ 0. , -0.5,  0. ]])

In [49]:
l

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [50]:
lu

array([[ 0.,  1.,  0.],
       [ 1.,  0., -1.],
       [ 0., -1.,  0.]])

In [91]:
X = np.linalg.eig(T)

In [90]:
p = np.amax(X[0])

In [93]:
2/(1 + math.sqrt(1 - pow(p, 2)))

---
### 5) Implemente um algoritmo que leia uma matriz A (de uma planilha excel) e:

In [52]:
def ler_matriz_A(file_matrix, sheet_matriz):
    df_matriz = pd.read_excel(r"dados/"+file_matrix+".xlsx", header=None, sheet_name=sheet_matriz) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    return matriz

matriz = ler_matriz_A("prova_ALC_testes", "matriz_A(2)")

**a)** Calcule a transposta de A;

**b)** Calcule o determinante de A;

**c)** Calcule a matriz adjunta de A;

**d)** Calcule o traço de A;

In [53]:
def traco(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    n, traco = min((linhas, colunas)), 0

    for i in range(n):
        traco += matriz_A[i,i]
    return traco

In [54]:
print("Traço de A:", traco(matriz))

Traço de A: 2024668.0


**e)** Avalie se A é simétrica;

In [55]:
def verifica_simetria(matriz_A):
    if verifica_quadrada(matriz_A):
        n = len(matriz_A)
        for i in range(n):
            for j in range(n):
                if (i != j) and (matriz_A[i,j] != matriz_A[j,i]):
                    print("A matriz não é simétrica")
                    return False
        print("A matriz é simétrica")
        return True
    print("A matriz não é quadrada")
    return False

In [56]:
print(verifica_simetria(matriz))

A matriz é simétrica
True


**f)** Avalie se A é ortogonal;

**g)** Calcule a norma de linha, norma de Frobenius e norma coluna de A.

In [57]:
def soma_linha(matriz_A): # norma-infinita matricial
    n, max, x = len(matriz_A), 0, 0

    for i in range(n):
        for j in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x

    return max

In [58]:
def frobenius(matriz_A): # norma-2 matricial
    n, x = len(matriz_A), 0

    for i in range(n):
        for j in range(n):
            x += math.pow(math.fabs(matriz_A[i,j]), 2)

    return x ** (1/2)

In [59]:
def soma_coluna(matriz_A): # norma-1 matricial
    n, max, x = len(matriz_A), 0, 0

    for j in range(n):
        for i in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x
    
    return max

In [60]:
print("Norma linha:", soma_linha(matriz))
print("Norma de Frobenius:", frobenius(matriz))
print("Norma coluna:", soma_coluna(matriz))

Norma linha: 37848742.0
Norma de Frobenius: 1544623.5421512907
Norma coluna: 37848742.0


**h)** Avalie se A é positiva definida.

---
### 6) Construa um algoritmo que receba dois vetores x e y e uma matriz A positiva definida (de uma planilha excel). Calcule:

In [61]:
def ler_matriz_A_vetor_x_vetor_y(file_matrix, file_array_x, file_array_y):
    df_matriz = pd.read_excel(r"dados/"+file_matrix+".xlsx", header=None) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor_x = pd.read_excel(r"dados/"+file_array_x+".xlsx", header=None) # Le o arquivo do excel que contém o vetor x sem cabeçalho
    df_vetor_y = pd.read_excel(r"dados/"+file_array_y+".xlsx", header=None) # Le o arquivo do excel que contém o vetor y sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor_x = np.ravel(np.array(df_vetor_x, dtype='f8')) # Converte o vetor b em um array da numpy
    vetor_y = np.ravel(np.array(df_vetor_y, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor_x, vetor_y


In [62]:
matriz, vetor_x, vetor_y = ler_matriz_A_vetor_x_vetor_y("matriz_A", "vetor_x", "vetor_y")

**a)** Norma 1, norma 2 e norma infinito de x e y.

In [63]:
def manhattan(vetor): # norma-1 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i])
    return x

In [64]:
def euclidiana(vetor): # norma-2 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i]) ** 2

    return x ** (1/2)

In [65]:
def infinita(vetor): # norma-infinita vetorial
    n, max = len(vetor), vetor[0]
    
    for i in range(1, n):
        if np.fabs(vetor[i]) > max:
            max = np.fabs(vetor[i])

    return max

In [66]:
print("Norma 1 do vetor x:", manhattan(vetor_x))
print("Norma 2 do vetor x:", euclidiana(vetor_x))
print("Norma infinita do vetor x:", infinita(vetor_x), "\n")
print("Norma 1 do vetor y:", manhattan(vetor_y))
print("Norma 2 do vetor y:", euclidiana(vetor_y))
print("Norma infinita do vetor y:", infinita(vetor_y))

Norma 1 do vetor x: 1004523.0
Norma 2 do vetor x: 269843.4839587571
Norma infinita do vetor x: 140094.0 

Norma 1 do vetor y: 1004523.0
Norma 2 do vetor y: 269843.4839587571
Norma infinita do vetor y: 140094.0


**b)** Ângulo entre x e y;

**c)** Norma de x e y induzida por A;

**d)** Produto interno de x e y.

---
### 7) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel). Encontre a fatoração LU de A.

In [67]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

# TESTE DO ALGORITMO DE FATORAÇÃO LU E NORMA RESIDUAL (fatoracaoLU.py, normas.py)'''
delta_x = fatoracaoLU(matriz,vetor)

print("Vetor x resultado da Fatoração LU:\n",delta_x,"\n")
#normas.residual(matriz, vetor, delta_x)

Vetor x resultado da Fatoração LU:
 [ 218.15690039 -134.91978861 -228.57366314  196.17807308  -23.10822144
  120.75613378   76.57658357   32.56339464  112.28655838 -311.76230605
    9.73618607  179.86106078   37.55533057   -5.28610392   20.3176418
  120.6584339   184.2906244    34.16170845 -178.63243142  -42.98537213
 -108.87178633  -70.23227857  168.6609164  -259.13412099 -164.019284  ] 



---
### 8) Construa um algoritmo que receba uma matriz A positiva (de uma planilha excel). Calcule:

In [68]:
matriz = ler_matriz_A("prova_ALC_testes", "matriz_A(2)")

**a)** O número condição;

**b)** Retorne uma mensagem ao usuário sobre sua interpretação.

---
### 9) Construa um algoritmo que receba três matrizes: U, sigma e V (de uma planilha excel) e:

In [69]:
def ler_matrizes_USV(file_excel):
    df_matriz_U = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_U') # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_matriz_S = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_S') # Le o arquivo do excel que contém o vetor x sem cabeçalho
    df_matriz_V = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_V') # Le o arquivo do excel que contém o vetor y sem cabeçalho
    matriz_U = np.array(df_matriz_U, dtype='f8') # Converte a matriz A em um array da numpy
    matriz_S = np.array(df_matriz_S, dtype='f8') # Converte a matriz A em um array da numpy
    matriz_V = np.array(df_matriz_V, dtype='f8') # Converte a matriz A em um array da numpy
    return matriz_U, matriz_S, matriz_V

matriz_U, matriz_S, matriz_V = ler_matrizes_USV("prova_ALC_testes")

**a)** Avalie se o produto das três matrizes representa uma decomposição SVD de alguma matriz X;

**b)** Caso a resposta à pergunta anterior seja negativa, retorne ao usuário uma explicação justificando esse fato.

---
### 10) Construa um algoritmo que leia uma planilha em Excel com 4 colunas e calcule a estatística de Benford (para 1 dígito) para cada uma das colunas.

In [70]:
benford(ler_dados_benford("dados_benford"))

Porcentagem de ocorrência de "1" em cada coluna: 10.6692% | 11.1430% | 11.8036% | 10.9420% | 
Porcentagem de ocorrência de "2" em cada coluna: 11.1574% | 11.0281% | 10.8271% | 11.3297% | 
Porcentagem de ocorrência de "3" em cada coluna: 11.3297% | 10.8415% | 11.3441% | 11.2005% | 
Porcentagem de ocorrência de "4" em cada coluna: 11.3584% | 10.5112% | 10.3676% | 10.7553% | 
Porcentagem de ocorrência de "5" em cada coluna: 10.6117% | 11.1430% | 11.2866% | 11.0999% | 
Porcentagem de ocorrência de "6" em cada coluna: 11.2435% | 11.3584% | 10.8845% | 11.0425% | 
Porcentagem de ocorrência de "7" em cada coluna: 11.8323% | 11.1861% | 11.4733% | 10.7266% | 
Porcentagem de ocorrência de "8" em cada coluna: 11.0999% | 11.1717% | 11.4159% | 11.6743% | 
Porcentagem de ocorrência de "9" em cada coluna: 10.6979% | 11.6169% | 10.5974% | 11.2005% | 
